In [0]:
pip install lightgbm

Python interpreter will be restarted.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 1.22.4 which is incompatible.
Python interpreter will be restarted.


In [0]:
pip install mlflow

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install xgboost

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install tensorflow

Python interpreter will be restarted.
  Using cached numpy-1.26.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.26.4 which is incompatible.
Python interpreter will be restarted.


In [0]:
%fs ls dbfs/FileStore/processed_data/


path,name,size,modificationTime
dbfs:/dbfs/FileStore/processed_data/STN_df_encoded.csv/,STN_df_encoded.csv/,0,0
dbfs:/dbfs/FileStore/processed_data/STN_df_encoded.parquet/,STN_df_encoded.parquet/,0,0


In [0]:
input_path_csv = '/dbfs/FileStore/processed_data/STN_df_encoded.csv'

# Load the CSV file into a Spark DataFrame
STN_spark_df_loaded_csv = spark.read.csv(input_path_csv, header=True, inferSchema=True)

# Convert the Spark DataFrame to a Pandas DataFrame
STN_df_encoded = STN_spark_df_loaded_csv.toPandas()

# Display the Pandas DataFrame
STN_df_encoded.head()

,PAT_ID,mpr,mpr80,MAX_GAP_IND,DS_nm,fills_nm,claim_90D_nm,claims_nm,percent_90D_nm,gpi10_cnt_nm,...,PREDOMINANT_RACE_BLACK,PREDOMINANT_RACE_HISPANIC,PREDOMINANT_RACE_OTHER RACE,PREDOMINANT_RACE_WHITE,URBANICITY_RURAL,URBANICITY_SUBURBAN,URBANICITY_URBAN,URBANICITY__Missing_,total_gaps,high_copay
0,1.094306e+10,1.000000,1,0,360,4,4,4,1.0,2,...,0,0,0,1,0,1,0,0,-23.0,0
1,1.094306e+10,1.000000,1,0,180,2,2,2,1.0,1,...,0,0,0,1,0,1,0,0,44.0,0
2,1.094307e+10,1.000000,1,0,270,3,3,3,1.0,2,...,0,0,0,1,0,1,0,0,2.0,0
3,1.094410e+10,0.642857,0,0,90,1,1,1,1.0,1,...,0,0,0,1,0,1,0,0,0.0,0
4,1.094411e+10,0.967532,1,0,180,2,2,2,1.0,1,...,0,0,0,1,0,1,0,0,45.0,0


### Predicting Medication Adherence

The primary goal of the model is to predict whether a patient is likely to adhere to their medication regimen. 

This is based on the mpr80 column, where:

• mpr80 = 1: Indicates that the patient's Medication Possession Ratio (MPR) is 80% or higher, meaning they are adherent to their medication.

• mpr80 = 0: Indicates that the patient's MPR is below 80%, meaning they are not adherent to their medication.

Identifying patients who are at high risk of non-adherence (i.e., those predicted to have mpr80 = 0).

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV


# Adding new features
STN_df_encoded['total_gaps'] = STN_df_encoded['gaps_bl'] + STN_df_encoded['leading_gap']
STN_df_encoded['high_copay'] = (STN_df_encoded['copay_tc_nm'] > STN_df_encoded['copay_tc_nm'].mean()).astype(int)

# Step 4: Define target and features
X = STN_df_encoded.drop(columns=['mpr', 'mpr80', 'PAT_ID'])  # Drop unnecessary columns including mpr and mpr80
y = STN_df_encoded['mpr80']  # Use mpr80 as the target variable

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

/databricks/python/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [0]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier


In [0]:
with mlflow.start_run(run_name="XGBoost Classifier STN"):
    # Initialize XGBoost Classifier
    xgb_model = XGBClassifier(random_state=42)
    
    # Train the model
    xgb_model.fit(X_train, y_train)
    
    # Predicting on test set
    y_pred = xgb_model.predict(X_test)
    y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("roc_auc", roc_auc)
    
    # Log the model
    mlflow.sklearn.log_model(xgb_model, "model")

    print(f"XGBoost - Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, ROC AUC Score: {roc_auc:.2f}")


/databricks/python/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/08/20 15:17:49 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.15.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/08/20 15:17:50 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/20 15:17:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBoost Classifier STN at: https://community.cloud.databricks.com/ml/experiments/2051286152915209/runs/5534184da99a44df9199393129bf2896.
2024/08/20 15:17:50 INFO mlflow.tracking._trackin

XGBoost - Accuracy: 0.97, Precision: 0.99, Recall: 0.98, F1 Score: 0.99, ROC AUC Score: 0.99


In [0]:
with mlflow.start_run(run_name="LightGBM Classifier STN"):
    # Initialize LGBM Classifier
    lgbm_model = LGBMClassifier(random_state=42)
    
    # Train the model
    lgbm_model.fit(X_train, y_train)
    
    # Predicting on test set
    y_pred = lgbm_model.predict(X_test)
    y_pred_proba = lgbm_model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("roc_auc", roc_auc)
    
    # Log the model
    mlflow.sklearn.log_model(lgbm_model, "model")

    print(f"LGBM - Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, ROC AUC Score: {roc_auc:.2f}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 35510, number of negative: 3529
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018713 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5351
[LightGBM] [Info] Number of data points in the train set: 39039, number of used features: 63
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.909603 -> initscore=2.308800
[LightGBM] [Info] Start training from score 2.308800


2024/08/20 15:17:57 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.15.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/08/20 15:17:57 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/20 15:17:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM Classifier STN at: https://community.cloud.databricks.com/ml/experiments/2051286152915209/runs/07bcebbe918941c18c9661504b6cfdde.
2024/08/20 15:17:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/2051286152915209.


LGBM - Accuracy: 0.98, Precision: 0.99, Recall: 0.98, F1 Score: 0.99, ROC AUC Score: 0.99


In [0]:
with mlflow.start_run(run_name="Logistic Regression STN"):
    # Initialize Logistic Regression
    logreg_model = LogisticRegression(random_state=42, max_iter=1000)
    
    # Train the model
    logreg_model.fit(X_train, y_train)
    
    # Predicting on test set
    y_pred = logreg_model.predict(X_test)
    y_pred_proba = logreg_model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("roc_auc", roc_auc)
    
    # Log the model
    mlflow.sklearn.log_model(logreg_model, "model")

    print(f"Logistic Regression - Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, ROC AUC Score: {roc_auc:.2f}")


/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2024/08/20 15:18:15 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.15.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/08/20 15:18:16 WARNING mlflow.models.model: Input example should be provided to infer model 

Logistic Regression - Accuracy: 0.91, Precision: 0.91, Recall: 1.00, F1 Score: 0.95, ROC AUC Score: 0.82


In [0]:
with mlflow.start_run(run_name="Random Forest Classifier STN"):
    # Initialize RandomForestClassifier
    rf_model = RandomForestClassifier(random_state=42)
    
    # Train the model
    rf_model.fit(X_train, y_train)
    
    # Predicting on test set
    y_pred = rf_model.predict(X_test)
    y_pred_proba = rf_model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("roc_auc", roc_auc)
    
    # Log the model
    mlflow.sklearn.log_model(rf_model, "model")

    print(f"Random Forest - Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, ROC AUC Score: {roc_auc:.2f}")


2024/08/20 15:18:27 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.15.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/08/20 15:18:29 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/20 15:18:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest Classifier STN at: https://community.cloud.databricks.com/ml/experiments/2051286152915209/runs/6c718dee6e5441f8b03f1f97a0aa7fd4.
2024/08/20 15:18:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/2051286152915209.


Random Forest - Accuracy: 0.98, Precision: 1.00, Recall: 0.98, F1 Score: 0.99, ROC AUC Score: 0.99


In [0]:
import mlflow.tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC

# Start an MLflow run
with mlflow.start_run(run_name="Neural Network Classifier STN"):
    # Define the neural network architecture
    model = Sequential([
        Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss=BinaryCrossentropy(),
                  metrics=['accuracy', AUC(name='auc')])

    # Train the model
    history = model.fit(X_train, y_train, 
                        epochs=10, 
                        batch_size=32, 
                        validation_split=0.2,
                        verbose=2)

    # Evaluate the model on the test set
    test_loss, test_accuracy, test_auc = model.evaluate(X_test, y_test, verbose=0)
    
    # Make predictions
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype("int32")

    # Calculate metrics
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log metrics
    mlflow.log_metric("accuracy", test_accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("roc_auc", test_auc)

    # Log the model
    mlflow.tensorflow.log_model(model, "model")

    print(f"Neural Network - Accuracy: {test_accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, ROC AUC Score: {test_auc:.2f}")


/local_disk0/.ephemeral_nfs/envs/pythonEnv-bc7d19f6-8bbc-4354-b6be-c796f63f9c54/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
976/976 - 4s - 5ms/step - accuracy: 0.8286 - auc: 0.4992 - loss: 565.6605 - val_accuracy: 0.9096 - val_auc: 0.5253 - val_loss: 0.3330
Epoch 2/10
976/976 - 5s - 5ms/step - accuracy: 0.8902 - auc: 0.5104 - loss: 7.0847 - val_accuracy: 0.9097 - val_auc: 0.5056 - val_loss: 0.3262
Epoch 3/10
976/976 - 2s - 2ms/step - accuracy: 0.9018 - auc: 0.4918 - loss: 2.2267 - val_accuracy: 0.9097 - val_auc: 0.5001 - val_loss: 0.3187
Epoch 4/10
976/976 - 3s - 3ms/step - accuracy: 0.9053 - auc: 0.5008 - loss: 1.0616 - val_accuracy: 0.9097 - val_auc: 0.5000 - val_loss: 0.3058
Epoch 5/10
976/976 - 2s - 2ms/step - accuracy: 0.9070 - auc: 0.4947 - loss: 0.6756 - val_accuracy: 0.9097 - val_auc: 0.5000 - val_loss: 0.3036
Epoch 6/10
976/976 - 2s - 2ms/step - accuracy: 0.9082 - auc: 0.4949 - loss: 0.4584 - val_accuracy: 0.9097 - val_auc: 0.5000 - val_loss: 0.3033
Epoch 7/10
976/976 - 3s - 3ms/step - accuracy: 0.9086 - auc: 0.5009 - loss: 0.4441 - val_accuracy: 0.9097 - val_auc: 0.5000 - val_loss: 0.30

2024/08/20 15:19:04 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/08/20 15:19:09 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.15.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/08/20 15:19:10 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/20 15:19:11 INFO mlflow.tracking._tracking_service.client: 🏃 View run Neural Network Classifier STN at: https://community.cloud.databricks.com/ml/experiments/20512861529152

Neural Network - Accuracy: 0.91, Precision: 0.91, Recall: 1.00, F1 Score: 0.95, ROC AUC Score: 0.50
